In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
from visualization import *
from util_tsne_exact import *
from sympy import *
init_printing()

## Calculating FLOPs per part

In [5]:
it, D, N, d, T = symbols('it D N d T')
count_measure = ("add", "mult", "div", "exp", "log")
flops = {
    "normalize": {
        "add": 2 * N * D,
        "div": D + N * D
    },
    "compute_pairwise_affinity_perplexity": {
        "compute_squared_euclidean_distance": {
            "add": D * N * (N - 1) / 2 * 2,
            "mult": D * N * (N - 1) / 2
        },
        "binary_search": {
            "add": it * (N + N + 1 + 1),
            "mult": it * (N + 2 * N),
            "div": it * (1 + 1) + N * N,
            "exp": it * N,
            "log": it * (1 + 1)
        }
    },
    "symmetrize_affinities": {
        "add": N * (N - 1) / 2 + N * N,
        "div": N * N
    },
    "early_exageration": {
        "mult": 2 * N * N
    },
    "compute_low_dimensional_affinities": {
        "compute_squared_euclidean_distance": {
            "add": T * d * N * (N - 1) / 2 * 2,
            "mult": T * d * N * (N - 1) / 2
        },
        "compute": {
            "add": T * N * (N - 1) * 2,
            "div": T * N * (N - 1)
        }
    },
    "gradient_computation": {
        "add": T * N * (N - 1) * (1 + 2 * d),
        "mult": T * N * (N - 1) * (1 + d),
        "div": T * N * (N - 1)
    },
    "gradient_update": {
        "add": T * (N * d + N * d * 2),
        "mult": T * N * d * 3
    },
    "normalize_2": {
        "add": T * 2 * N * d,
        "div": T * (d + N * d)
    }
}

flops_by_function_measure = dict(flops)


In [6]:
cost_part_1 = {}
cost_part_2 = {}
cost_part_3 = {}

for c in flops_by_function_measure["compute_pairwise_affinity_perplexity"]["compute_squared_euclidean_distance"]:
    if c not in cost_part_1:
        cost_part_1[c] = 0
    cost_part_1[c] += flops_by_function_measure["compute_pairwise_affinity_perplexity"]["compute_squared_euclidean_distance"][c]
    
for c in flops_by_function_measure["normalize"]:
    if c not in cost_part_1:
        cost_part_1[c] = 0
    cost_part_1[c] += flops_by_function_measure["normalize"][c]
    
    
for c in flops_by_function_measure["compute_pairwise_affinity_perplexity"]["binary_search"]:
    if c not in cost_part_2:
        cost_part_2[c] = 0
    cost_part_2[c] += flops_by_function_measure["compute_pairwise_affinity_perplexity"]["binary_search"][c]

for c in flops_by_function_measure["symmetrize_affinities"]:
    if c not in cost_part_2:
        cost_part_2[c] = 0
    cost_part_2[c] += flops_by_function_measure["symmetrize_affinities"][c]
    
    
for c in flops_by_function_measure["early_exageration"]:
    if c not in cost_part_2:
        cost_part_2[c] = 0
    cost_part_2[c] += flops_by_function_measure["early_exageration"][c]/2

    
for t in ["gradient_computation", "gradient_update", "normalize_2"]:
    for c in flops_by_function_measure[t]:
        if c not in cost_part_3:
            cost_part_3[c] = 0
        cost_part_3[c] += flops_by_function_measure[t][c]/2

        
for c in flops_by_function_measure["early_exageration"]:
    if c not in cost_part_3:
        cost_part_3[c] = 0
    cost_part_3[c] += flops_by_function_measure["early_exageration"][c]/2

        
print("Part 1")
for c in cost_part_1:
    print(c, cost_part_1[c].simplify())

print()
print("Part 2")
for c in cost_part_2:
    print(c, cost_part_2[c].simplify(), "- -", cost_part_2[c].subs('it', 20*N).simplify())
    
print()
print("Part 3")
for c in cost_part_3:
    print(c, cost_part_3[c].simplify())

Part 1
add D*N*(N + 1)
mult D*N*(N - 1)/2
div D*(N + 1)

Part 2
add 3*N**2/2 + 2*N*it - N/2 + 2*it - - N*(83*N + 79)/2
mult N*(N + 3*it) - - 61*N**2
div 2*N**2 + 2*it - - 2*N*(N + 20)
exp N*it - - 20*N**2
log 2*it - - 40*N

Part 3
add N*T*(5*d + (N - 1)*(2*d + 1))/2
mult N*(2*N + 3*T*d + T*(N - 1)*(d + 1))/2
div T*(N*(N - 1) + d*(N + 1))/2


### TODO use a combined cost measure where div's are more expensive

## Time spend per part Based on D

In [29]:
d500 = pd.read_csv("../code/implementations/tsne_exact_final/toaster_bench/D500")
d1000 = pd.read_csv("../code/implementations/tsne_exact_final/toaster_bench/D1000")
d2500 = pd.read_csv("../code/implementations/tsne_exact_final/toaster_bench/D2500")

In [30]:
part1_500 = d500["pairwise_squared_euclidean_distance"]
part2_500 = d500["pairwise_affinity_perplexity"] + d500["symmetrize_affinities"]
part3_500 = d500["low_dimensional_affinities"] + d500["gradient_computation_update_normalize"]
tot = d500["pairwise_squared_euclidean_distance"] + d500["pairwise_affinity_perplexity"] + d500["symmetrize_affinities"] + d500["low_dimensional_affinities"] + d500["gradient_computation_update_normalize"]
part1_500 /= tot
part2_500 /= tot
part3_500 /= tot
part1_500 = part1_500[0]
part2_500 = part2_500[0]
part3_500 = part3_500[0]
print(part1_500, part2_500, part3_500)

0.0448644654934 0.180571933685 0.774563600821


In [31]:
part1_1000 = d1000["pairwise_squared_euclidean_distance"]
part2_1000 = d1000["pairwise_affinity_perplexity"] + d1000["symmetrize_affinities"]
part3_1000 = d1000["low_dimensional_affinities"] + d1000["gradient_computation_update_normalize"]
tot = d1000["pairwise_squared_euclidean_distance"] + d1000["pairwise_affinity_perplexity"] + d1000["symmetrize_affinities"] + d1000["low_dimensional_affinities"] + d1000["gradient_computation_update_normalize"]
part1_1000 /= tot
part2_1000 /= tot
part3_1000 /= tot
part1_1000 = part1_1000[0]
part2_1000 = part2_1000[0]
part3_1000 = part3_1000[0]
print(part1_1000, part2_1000, part3_1000)

0.0884400224532 0.172425723104 0.739134254443


In [32]:
part1_2500 = d2500["pairwise_squared_euclidean_distance"]
part2_2500 = d2500["pairwise_affinity_perplexity"] + d2500["symmetrize_affinities"]
part3_2500 = d2500["low_dimensional_affinities"] + d2500["gradient_computation_update_normalize"]
tot = d2500["pairwise_squared_euclidean_distance"] + d2500["pairwise_affinity_perplexity"] + d2500["symmetrize_affinities"] + d2500["low_dimensional_affinities"] + d2500["gradient_computation_update_normalize"]
part1_2500 /= tot
part2_2500 /= tot
part3_2500 /= tot
part1_2500 = part1_2500[0]
part2_2500 = part2_2500[0]
part3_2500 = part3_2500[0]
print(part1_2500, part2_2500, part3_2500)

0.233251350304 0.172769192406 0.59397945729


,N,pairwise_squared_euclidean_distance,pairwise_affinity_perplexity,symmetrize_affinities,low_dimensional_affinities,gradient_computation_update_normalize,total
0,1000,5.039704e+09,3.723792e+09,9.115551e+06,8.740625e+09,4.093088e+09,4.318419e+10
